<p style="color:#008bcb; font-family:Arial; font-size:19px;"> Notebook Information </p>

<table style="color:rgb(88,89,91); font-family:Arial; float:left; font-size:13px; text-align:left;">
<tr>
<td style="color:#008BCB;font-size:13px; text-align:left;"><b>Report</b></td>
<td style="text-align:left;">Exploratory Data Analysis and Automated ETL pipeline for PowerBI Audit Log </td>
</tr>
<tr>
<td style="color:#008BCB;font-size:13px; text-align:left;"><b>Author</b></td>
<td style="text-align:left;">Sanmi Ibitoye</td>
</tr>
<tr>
<td style="color:#008BCB;font-size:13px; text-align:left;"><b>Email</b></td>
<td style="text-align:left;">Sanmi.Ibitoye@hs2.org.uk</td>
</tr>
<tr>
<td style="color:#008BCB;font-size:13px; text-align:left;"><b>Summary</b></td>
<td style="text-align:left;">This notebook forms the basis of all future etl, tools and infrastructure doployable in relation to PowerBI Audit Log Report project. <br>
Initial analysis and exploration helps unearth the data so that its shape, format and condition is understood by team memebers and explainable to stakeholders. <br>
The resulting pipelines and infrastructure will employ the use of technology stacks such as git, postgres, docker, dbt, bespoke python libraries, airbyte and <br> prefect for automation, ochestration and compute. Alternative stack could also be implemented<br>
This will serve a powerbi dashboard data for its vizualisation and translation to english for the stakeholder <br>
</td>
</tr>
<tr>
<td style="color:#008BCB;font-size:13px; text-align:left;"><b>Date</b></td>
<td style="text-align:left;">Last Updated: 08 - 12 - 2022 </td>
</tr>

</table>

In [ ]:
# Define the required library Imports
try:
    import pandas as pd
    import numpy as np
    from functools import wraps
    import os
    import glob

    import json
    import math

    # import dask
    # from dask.distributed import Client
    # import dask.dataframe as dd
    # import dask.multiprocessing
    import yaml
    import base64

    from IPython.display import display, HTML

    # display(HTML("<style>.container {width:90% !important;}<style/>"))

    # datetime libraries
    import datetime
    from datetime import datetime, timedelta

    from functools import wraps
    from calendar import monthrange
    import datetime as dt
    import time

    # prefect libraries
    from prefect import task, Flow

    # from prefect.schedules import IntervalSchedule

    # plotting libraries
    import seaborn as sns
    import matplotlib.pyplot as plt

    # profiling libraries
    # from pandas_profiling import ProfileReport

    # config libraries
    import hydra
    from omegaconf import DictConfig, OmegaConf
    from hydra import utils


except Exception as e:
    print("Some Modules are Missing : {} ".format(e))

<p style="color:#008bcb; font-family:Arial; font-size:19px;">1 - Turn multiple csv into a single pandas dataframe.</p>
<tr>
<td style="color:#008BCB;font-size:13px; text-align:left;"><b>Note 1.1 </b></td>
<td style="color:rgb(88,89,91);font-size:13px; text-align:left;">The Power BI Activily Log data is extracted using a powershell script that only allows an API call of maximum 30 days from current date. <br>
A single csv file holds the data for each day of selected columns defined in the powershell script. Each day represented by a csv file is stacked into a single pandas dataframe prior to subsequent analysis</td>
</tr>

<tr>
<td style="color:#008BCB;font-size:13px; text-align:left;"><b>Note 1.2 </b></td>
<td style="color:rgb(88,89,91);font-size:13px; text-align:left;">The CSV files are obtained using a shell script. Although atleast 4 methods of interacting with the API exists, the option used in this solution fetches satisfies all possible user requirements known now and any unkown in the future <br>
The 74 columns extracted holds a set of operations, all of which fall within the common activities that 98% of Power BI users will execute.<br> Analysis of the raw data has shown that the other operations which are rare and not relevant to the audit purpose often come through as blanks.</td>
</tr>

<tr>
<td style="color:#008BCB;font-size:13px; text-align:left;"><b>Note 1.3 </b></td>
<td style="color:rgb(88,89,91);font-size:13px; text-align:left;">Using glob.glob works differently on Ubuntu compared to Windows <br>
The file location is passed as a variable so when configuring or debugging, attention should be given to glob.glob pattern that applies to host system</td>
</tr>

<tr>
<td style="color:#008BCB;font-size:13px; text-align:left;">Ref. Bash Script</td>
<td style="color:rgb(88,89,91);font-size:13px; text-align:left;">https://community.powerbi.com/t5/Service/User-activities-by-Powershell/m-p/2071415</td>
</tr>

In [ ]:
# import helper from src folder
from mlops_pbi.src.helper import load_config

# load raw files and concat into single data frame using columns defined above as column names for the dataframe to be created.
@hydra.main(config_path="config", config_name="main", version_base="1.2")
def load_raw_files(config: DictConfig) -> pd.DataFrame:
    """
    Load raw files and concat into single data frame using columns defined above as column names for the dataframe to be created.
    """
    all_df = pd.concat(
        [
            pd.read_csv(
                one_filename,
                low_memory=config.raw_data.low_memory,
                header=config.raw_data.header,
                sep=",",
                encoding=config.raw_data.encoding,
                names=config.raw_data.names,
            )
            for one_filename in glob.glob(config.raw_data.path)
        ]
    )

    return all_df


# assign function to a variable and loads dataframe into memory for subsequent analysis with the specified columns

config = load_config()
df = load_raw_files(config)

# df.columns = columns

In [ ]:
df.head()

In [ ]:
# optional - save dataframe to csv
# df_raw_profile = ProfileReport(df, title="PowerBI AuditLog Raw Data Profiling Report", explorative=True)
# df_raw_profile.to_file("df_raw_profile.html")

<p style="color:#008bcb; font-family:Arial; font-size:19px;">2 - Run transformation pipeline on multiple csv concatenated into a single pandas dataframe.</p>
<tr>
<td style="color:#008BCB;font-size:13px; text-align:left;"><b>Note 2.1 </b></td>
<td style="color:rgb(88,89,91);font-size:13px; text-align:left;">The Power BI Activily Log data extracted using a powershell script has now been concatenated into a single dataframe. <br>
A single dataframe holds all the data in memory. The tansformation pipeline makes a copy of the dataframe to avoid mutations on the original copy. <br> The pipeline is then run on the copy of the dataframe and the original dataframe is left untouched. <br> Date columns are converted to datetime format. <br> The datetime column is used to extract the month, year and day.
</td>
</td>
</td>
</tr>

<tr>
<td style="color:#008BCB;font-size:13px; text-align:left;"><b>Note 2.2 </b></td>
<td style="color:rgb(88,89,91);font-size:13px; text-align:left;">The raw data contains logs that relate to the service account used to fetch the data. These logs skew the data and don't represent a user's activity. <br> As such, the last step in the initial transformation is to filter such logs out. <br> The service account API calls are logged with a unique reference of multiple 0 values. This values is passed as a parameter for the filtering out of all logs not related to an actual user. <br>
The 36 columns extracted holds a set of operations, all of which fall within the common activities that 98% of Power BI users will execute.<br> Analysis of the raw data has shown that the other operations which are rare and not relevant to the audit purpose often come through as blanks.</td>
</tr>

<tr>
<td style="color:#008BCB;font-size:13px; text-align:left;"><b>Note 2.3 </b></td>
<td style="color:rgb(88,89,91);font-size:13px; text-align:left;">This first node in the pipeline is stored as variable "use_df" <br>
</td>
</tr>

<tr>
<td style="color:#008BCB;font-size:13px; text-align:left;">Ref. Bash Script</td>
<td style="color:rgb(88,89,91);font-size:13px; text-align:left;">https://community.powerbi.com/t5/Service/User-activities-by-Powershell/m-p/2071415</td>
</tr>

In [ ]:
# import python functions to build pipeline
from mlops_pbi.src.first_ingestion_point import (
    start_pipeline_pbia,
    format_datatype_dates,
    add_month_year_cols_users,
    api_calls_filtered_out,
    lower_case_cols,
)

In [ ]:
# variables for the 4th transformation node i.e. api_calls_filtered_out
service_account = "SA_DAL_PowerBI@hs2.org.uk"
operation = "ExportActivityEvents"


use_df = (
    df.pipe(start_pipeline_pbia)
    .pipe(format_datatype_dates)
    .pipe(add_month_year_cols_users)
    .pipe(api_calls_filtered_out, service_account, operation)
    .pipe(lower_case_cols, "UserId", "ReportName")
)

# singleUserLogic = use_df.pipe()

<p style="color:#008bcb; font-family:Arial; font-size:19px;">3 - Run transformation pipeline on fist node i.e. use_df.</p>
<tr>
<td style="color:#008BCB;font-size:13px; text-align:left;"><b>Note 3.1 </b></td>
<td style="color:rgb(88,89,91);font-size:13px; text-align:left;">This second node in the transformation pipeline extracts useful features for better understanding of the data and machine learning experiments <br> The second node adds new columns to the data set. These columns make it possible to information related to one or multiple entities. <br> length of days with activity, length of days since last active, length of days since first active are all new coloumns derived from time delta. <br> These 3 new columns allows a measure of frequency and recency to be 2 extra features added to the dataframe.
</td>
</td>
</td>
</tr>

<tr>
<td style="color:#008BCB;font-size:13px; text-align:left;"><b>Note 3.2 </b></td>
<td style="color:rgb(88,89,91);font-size:13px; text-align:left;">
The second node ends the transformation pipeline. The dataframe is now ready for analysis and machine learning experiments. The output of both nodes are extracted as csv files. <br> This csv file forms the source data for the unsupervised machine learning process used to cluster the users into 4 quadrants.  <br>
</td>
</tr>

<tr>
<td style="color:#008BCB;font-size:13px; text-align:left;"><b>Note 3.3 </b></td>
<td style="color:rgb(88,89,91);font-size:13px; text-align:left;">The second node is stored as variable "use_df_2" <br>
</td>
</tr>

<tr>
<td style="color:#008BCB;font-size:13px; text-align:left;">Ref. Bash Script</td>
<td style="color:rgb(88,89,91);font-size:13px; text-align:left;">https://community.powerbi.com/t5/Service/User-activities-by-Powershell/m-p/2071415</td>
</tr>

In [ ]:
use_df.info()

In [ ]:
from mlops_pbi.src.first_ingestion_point import (
    lengthOfDays_since_firstRecord,
    lengthOfDays_since_lastActive,
    lengthOfDays_withActivity,
    single_user_df,
    single_user_frequency,
    percent_time_inactiveFor,
)

In [ ]:
use_df_2 = (
    use_df.pipe(single_user_df)
    .pipe(lengthOfDays_withActivity)
    .pipe(lengthOfDays_since_lastActive)
    .pipe(lengthOfDays_since_firstRecord)
    .pipe(single_user_frequency)
    .pipe(percent_time_inactiveFor)
)

In [ ]:
use_df_2

In [ ]:
from mlops_pbi.src.first_ingestion_point import (
    compare_df_col,
    blank_users_df,
    save_missing_users_df,
)

In [ ]:
# compare the 2 dataframes to see if any users are missing from the second node
missing_users_list = compare_df_col(use_df, use_df_2, "UserId")

missing_users_df = blank_users_df(use_df, missing_users_list)

file_path_missing_users = r"C:\Users\sibitoye\Documents\PBI_AuditLog_Final_Python_Output\missing_PBI_Clean_July_August_2022.csv"
save_missing_users_df(df=missing_users_df, path=file_path_missing_users)

In [ ]:
# function to save use_df_2 to csv
def save_df_to_csv(df, path):
    df.to_csv(path, index=False)

In [ ]:
path_to_saveto = r"C:\Users\sibitoye\Documents\PBI_AuditLog_Final_Python_Output\PBI_Clean_2022.csv"
save_df_to_csv(df=use_df_2, path=path_to_saveto)

In [ ]:
use_df_2.describe(include="O", datetime_is_numeric=True).T

In [ ]:
use_df_2.describe(exclude="O", datetime_is_numeric=True).T

In [ ]:
use_df_2

<p style="color:#008bcb; font-family:Arial; font-size:19px;">4 - Data Visualisation</p>

In [ ]:
from sklearn.cluster import k_means
from sklearn.preprocessing import MinMaxScaler
from scipy import stats
from scipy.stats import pearsonr

In [ ]:
# use_df_2_for_ds = use_df_2.loc[use_df_2["%_time_inactiveFor"] != "NaN"]# use_df_2[use_df_2['frequency'] == 'Nan'])
use_df_2_for_ds = use_df_2.replace([np.inf, -np.inf], np.nan).dropna(axis=0)

In [ ]:
# use_df_2_for_ds = use_df_2.dropna()

In [ ]:
use_df_2_for_ds.describe().T

In [ ]:
corr, _ = pearsonr(
    use_df_2_for_ds["%_time_inactiveFor"], use_df_2_for_ds["frequency"]
)
jointPlot = (
    sns.jointplot(
        x="%_time_inactiveFor",
        y="frequency",
        data=use_df_2_for_ds,
        kind="reg",
        height=10,
    )
).plot_joint(sns.kdeplot, zorder=0, n_levels=6)

plt.show()

In [ ]:
corr, _ = pearsonr(
    use_df_2_for_ds["lengthOfDays_since_lastActive"],
    use_df_2_for_ds["frequency"],
)
jointPlot = (
    sns.jointplot(
        x="lengthOfDays_since_lastActive",
        y="frequency",
        data=use_df_2_for_ds,
        kind="reg",
        height=10,
    )
).plot_joint(sns.kdeplot, zorder=0, n_levels=6)

plt.show()

In [ ]:
columns = [
    "%_time_inactiveFor",
    "lengthOfDays_withActivity",
    "lengthOfDays_since_lastActive",
    "lengthOfDays_since_firstRecord",
    "frequency",
]


plt.figure(1, figsize=(25, 12))
n = 0
for x in [
    "%_time_inactiveFor",
    "lengthOfDays_withActivity",
    "lengthOfDays_since_lastActive",
    "lengthOfDays_since_firstRecord",
    "frequency",
]:
    n += 1
    plt.subplot(3, 3, n)
    plt.subplots_adjust(hspace=0.5, wspace=0.5)
    sns.histplot(use_df_2_for_ds[x], bins=15)
    plt.title("Distplot of {}".format(x))
plt.show()

In [ ]:
sns.pairplot(use_df_2_for_ds[columns], hue="%_time_inactiveFor", aspect=2)
plt.show()

<p style="color:#008bcb; font-family:Arial; font-size:19px;">5 - Machine Learning Section - Clustering</p>

In [ ]:
dscols = [
    "UserId",
    "lengthOfDays_withActivity",
    "frequency",
    "%_time_inactiveFor",
]  #'lengthOfDays_since_lastActive','lengthOfDays_since_firstRecord',

In [ ]:
# use clustering analysis on dataframe to determine number of clusters to use
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from scipy import stats
from scipy.stats import pearsonr
from sklearn.preprocessing import LabelEncoder

In [ ]:
use_df_2_for_ds[dscols]

In [ ]:
df = use_df_2_for_ds[dscols]
X = df

y = X["UserId"]

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

X["UserId"] = le.fit_transform(X["UserId"])

y = le.transform(y)

In [ ]:
# X[y]

In [ ]:
# feature scaling
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

In [ ]:
# transform X to dataframe
X = pd.DataFrame(X, columns=dscols)

In [ ]:
# kmeans clustering
kmeans = KMeans(n_clusters=7, random_state=0).fit(X)

In [ ]:
# check model parameters
kmeans.cluster_centers_



    The KMeans algorithm clusters data by trying to separate samples in n groups of equal variances, minimizing a criterion known as inertia, or within-cluster sum-of-squares Inertia, or the within-cluster sum of squares criterion, can be recognized as a measure of how internally coherent clusters are.

    The k-means algorithm divides a set of N samples X into K disjoint clusters C, each described by the mean j of the samples in the cluster. The means are commonly called the cluster centroids.

    The K-means algorithm aims to choose centroids that minimize the inertia, or within-cluster sum of squared criterion.

Inertia

    Inertia is not a normalized metric.

    The lower values of inertia are better and zero is optimal.

    But in very high-dimensional spaces, euclidean distances tend to become inflated (this is an instance of curse of dimensionality).

    Running a dimensionality reduction algorithm such as PCA prior to k-means clustering can alleviate this problem and speed up the computations.

    We can calculate model inertia as follows:-




In [ ]:
# check model internal consistency
kmeans.inertia_



    The lesser the model inertia, the better the model fit.

    We can see that the model has reasonably low inertia. So, this might be a good model fit to the data.



In [ ]:
# check quality of model fit
labels = kmeans.labels_

# check how many of the samples were correctly labeled
correct_labels = sum(y == labels)

print(
    "Result: %d out of %d samples were correctly labeled."
    % (correct_labels, y.size)
)

In [ ]:
# elbow method to determine optimal number of clusters
from sklearn.cluster import KMeans

cs = []
for i in range(1, 11):
    kmeans = KMeans(
        n_clusters=i, init="k-means++", max_iter=300, n_init=10, random_state=0
    )
    kmeans.fit(X)
    cs.append(kmeans.inertia_)
plt.plot(range(1, 11), cs)
plt.title("The Elbow Method")
plt.xlabel("Number of clusters")
plt.ylabel("inertia")
plt.show()

In [ ]:
# kmeans clustering with optimal number of clusters
kmeans = KMeans(
    n_clusters=8,
    init="k-means++",
    n_init=10,
    max_iter=300,
    tol=0.0001,
    random_state=111,
    algorithm="elkan",
).fit(X)

In [ ]:
kmeans.inertia_

In [ ]:
# check quality of model fit
labels = kmeans.labels_

# check how many of the samples were correctly labeled
correct_labels = sum(y == labels)

print(
    "Result: %d out of %d samples were correctly labeled."
    % (correct_labels, y.size)
)

In [ ]:
# check how many of the samples were correctly labeled
correct_labels = sum(y == labels)

In [ ]:
# show cluster centroids
kmeans.cluster_centers_

In [ ]:
# Create cluster feature
kmeans = KMeans(n_clusters=6)
X["Cluster"] = kmeans.fit_predict(X)
X["Cluster"] = X["Cluster"].astype("category")

X

In [ ]:
# # columns of interest
# cols = ['Id', 'CreationTime', 'CreationTimeUTC', 'RecordType', 'Operation', 'OrganizationId', 'UserType', 'UserKey', 'Workload', 'UserId', 'ClientIP', 'UserAgent', 'Activity', 'ItemName', 'WorkSpaceName', 'DashboardName', 'DatasetName', 'ReportName',
# 'WorkspaceId', 'ObjectId', 'DashboardId', 'DatasetId', 'ReportId', 'OrgAppPermission', 'CapacityId', 'CapacityName', 'AppName', 'IsSuccess', 'ReportType', 'RequestId', 'ActivityId', 'AppReportId', 'DistributionMethod', 'ConsumptionMethod', 'RetrieveDate']

In [ ]:
algorithm = KMeans(
    n_clusters=4,
    init="k-means++",
    n_init=10,
    max_iter=300,
    tol=0.0001,
    random_state=111,
    algorithm="elkan",
)
algorithm.fit(X)
labels1 = algorithm.labels_
centroids1 = algorithm.cluster_centers_

In [ ]:
centroids1

In [ ]:
# fileToSave = load_raw_files(allcsv_windows_glob_glob)

In [ ]:
# # function to write csv file to disk
# def save_df_to_disk(df, path) -> pd.DataFrame:
#     """
#
#     :param df:
#     :param path:
#     :return:
#     """
#     df.to_csv(path, index=False)
#     return df

In [ ]:
# save_df_to_disk(diff_in_days, r'C:\Users\sibitoye\Documents\PBI_AuditLog_Final_Python_Output\PBI_Clean_July_August_2022.csv')

In [ ]:
# export use_df_2_for_ds to csv
# use_df_2.to_csv(r'C:\Users\sibitoye\Documents\PBI_AuditLog_Final_Python_Output\PBI_Clean_July_August_2022.csv', index=False)

In [ ]:
# # function to write csv file to disk
# def save_df_to_disk(df, path) -> pd.DataFrame:
#     """
#
#     :param df:
#     :param path:
#     :return:
#     """
#     df.to_csv(path, index=False)
#     return df

# UNSUPERVISED MACHINE LEARNING EXPERIMENT - K-MEANS CLUSTERING

In [ ]:
# set up the dataframe and fit it to kmeans clustering algorithm

X1 = use_df_2_for_ds[["frequency", "%_time_inactiveFor"]].iloc[:, :].values
inertia = []
for n in range(1, 15):
    algorithm = KMeans(
        n_clusters=n,
        init="k-means++",
        n_init=10,
        max_iter=300,
        tol=0.0001,
        random_state=111,
        algorithm="lloyd",
    )
    algorithm.fit(X1)
    inertia.append(algorithm.inertia_)

In [ ]:
# check the optimum number of clusters to give the best model performance
plt.figure(1, figsize=(15, 6))
plt.plot(np.arange(1, 15), inertia, "o")
plt.plot(np.arange(1, 15), inertia, "-", alpha=0.5)
plt.xlabel("Number of Clusters"), plt.ylabel("Inertia")
plt.show()

# UNSUPERVISED MACHINE LEARNING EXPERIMENT WITH 4 CLUSTERS

In [ ]:
algorithm = KMeans(
    n_clusters=4,
    init="k-means++",
    n_init=10,
    max_iter=300,
    tol=0.0001,
    random_state=111,
    algorithm="elkan",
)
algorithm.fit(X1)
labels1 = algorithm.labels_
centroids1 = algorithm.cluster_centers_

In [ ]:
h = 0.02
x_min, x_max = X1[:, 0].min() - 1, X1[:, 0].max() + 1
y_min, y_max = X1[:, 1].min() - 1, X1[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
Z = algorithm.predict(np.c_[xx.ravel(), yy.ravel()])

In [ ]:
plt.figure(1, figsize=(15, 7))
plt.clf()
Z = Z.reshape(xx.shape)
plt.imshow(
    Z,
    interpolation="nearest",
    extent=(xx.min(), xx.max(), yy.min(), yy.max()),
    cmap=plt.cm.Pastel2,
    aspect="auto",
    origin="lower",
)

plt.scatter(x="frequency", y="%_time_inactiveFor", data=df, c=labels1, s=100)
plt.scatter(x=centroids1[:, 0], y=centroids1[:, 1], s=300, c="red", alpha=0.5)
plt.ylabel("%_time_inactiveFor"), plt.xlabel("frequency")
plt.show()

In [ ]:
algorithm.inertia_

# UNSUPERVISED MACHINE LEARNING EXPERIMENT WITH 5 CLUSTERS

In [ ]:
algorithm = KMeans(
    n_clusters=5,
    init="k-means++",
    n_init=10,
    max_iter=300,
    tol=0.0001,
    random_state=111,
    algorithm="lloyd",
)
algorithm.fit(X1)
labels1 = algorithm.labels_
centroids1 = algorithm.cluster_centers_

In [ ]:
h = 0.02
x_min, x_max = X1[:, 0].min() - 1, X1[:, 0].max() + 1
y_min, y_max = X1[:, 1].min() - 1, X1[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
Z = algorithm.predict(np.c_[xx.ravel(), yy.ravel()])

In [ ]:
plt.figure(1, figsize=(15, 7))
plt.clf()
Z = Z.reshape(xx.shape)
plt.imshow(
    Z,
    interpolation="nearest",
    extent=(xx.min(), xx.max(), yy.min(), yy.max()),
    cmap=plt.cm.Pastel2,
    aspect="auto",
    origin="lower",
)

plt.scatter(x="frequency", y="%_time_inactiveFor", data=df, c=labels1, s=100)
plt.scatter(
    x=centroids1[:, 0], y=centroids1[:, 1], s=300, c="blue", alpha=0.75
)
plt.ylabel("%_time_inactiveFor"), plt.xlabel("frequency")
plt.show()

In [ ]:
algorithm.inertia_

# 3 Dimensional Clustering

In [ ]:
X3 = (
    df[["frequency", "%_time_inactiveFor", "lengthOfDays_withActivity"]]
    .iloc[:, :]
    .values
)
inertia = []
for n in range(1, 11):
    algorithm = KMeans(
        n_clusters=n,
        init="k-means++",
        n_init=10,
        max_iter=300,
        tol=0.0001,
        random_state=111,
        algorithm="lloyd",
    )
    algorithm.fit(X3)
    inertia.append(algorithm.inertia_)

In [ ]:
plt.figure(1, figsize=(15, 6))
plt.plot(np.arange(1, 11), inertia, "o")
plt.plot(np.arange(1, 11), inertia, "-", alpha=0.5)
plt.xlabel("Number of Clusters"), plt.ylabel("Inertia")
plt.show()

In [ ]:
algorithm = KMeans(
    n_clusters=7,
    init="k-means++",
    n_init=10,
    max_iter=300,
    tol=0.0001,
    random_state=111,
    algorithm="lloyd",
)
algorithm.fit(X3)
labels3 = algorithm.labels_
centroids3 = algorithm.cluster_centers_

y_kmeans = algorithm.fit_predict(X3)

In [ ]:
# check quality of model fit
algorithm.inertia_

In [ ]:
df["cluster"] = pd.DataFrame(y_kmeans)
df.head()

In [ ]:
# import plotly as py
# import plotly.graph_objs as go

# trace1 = go.Scatter3d(
#     x= df['frequency'],
#     y= df['lengthOfDays_withActivity'],
#     z= df['%_time_inactiveFor'],
#     mode='markers',
#      marker=dict(
#         color = df['cluster'],
#         size= 2,
#         line=dict(
#             color= df['cluster'],
#             width= 10
#         ),
#         opacity=0.8
#      )
# )
# data = [trace1]
# layout = go.Layout(
#     title= 'Clusters wrt frequency, lengthOfDays and %_time_inactiveFor',
#     scene = dict(
#             xaxis = dict(title  = 'frequency'),
#             yaxis = dict(title  = 'lengthOfDays'),
#             zaxis = dict(title  = '%_time_inactiveFor')
#         )
# )
# fig = go.Figure(data=data, layout=layout)
# py.offline.iplot(fig)

In [ ]:
use_df_2_for_ds["cluster"] = pd.DataFrame(y_kmeans)
# use_df_2_for_ds.head(20)

In [ ]:
use_df_2_for_ds.describe().T

In [ ]:
# plot distribution of clusters
sns.catplot(
    x="cluster",
    y="%_time_inactiveFor",
    data=use_df_2_for_ds,
    kind="box",
    aspect=1.5,
    height=9,
)

In [ ]:
# plot distribution of clusters
sns.catplot(
    x="cluster",
    y="lengthOfDays_withActivity",
    data=use_df_2_for_ds,
    kind="bar",
    aspect=1.5,
    height=9,
)

# -> we can see that the clusters are not very well separated
# -> people in cluster 1 and 5 have more time with activity than people in cluster 2 and 3
# -> people in cluster 2 and 3 are neutral compared to people in cluster 1 and 5 AND cluster 4, 6 and 0
# -> so it is easy to conclude that cluster 1 and 5 are the most active people i.e. Great!
# -> clusters 4, 6 and 0 are the least active people i.e. Not Great!
# -> clusters 2 and 3 are neutral people i.e. Neutral!

In [ ]:
# plot seaborn clustermap
sns.clustermap(use_df_2.corr(), annot=True, cmap="coolwarm", figsize=(20, 15))

<p style="color:#008bcb; font-family:Arial; font-size:19px;">6 - Export Final Output </p>

In [ ]:
# divide customers into 6 segments based on outcome of clustering analysis
def four_quad_segment(use_df_2):
    """
    This function takes in a dataframe and returns a dataframe with a new column.
    The new column is a segment based on the frequency and %_time_inactiveFor.

    :param use_df_2: df
    :return: df

    """
    if use_df_2["frequency"] <= 30 and use_df_2["%_time_inactiveFor"] <= 65:
        return "Great - Good Frequency, Good Recency"
    elif use_df_2["frequency"] <= 30 and use_df_2["%_time_inactiveFor"] > 65:
        return "Good - Good Frequency, Bad Recency"
    elif use_df_2["frequency"] > 30 and use_df_2["%_time_inactiveFor"] <= 65:
        return "Fair - Bad Frequency, Good Recency"
    else:
        return "Investigate - Bad Frequency, Bad Recency"


use_df_2["segment"] = use_df_2.apply(four_quad_segment, axis=1)

In [ ]:
use_df_2.head()

In [ ]:
# use_df_2.to_csv(r'/home/sanmi/OlaJay/SimpleAnonlify/PBI_Clean_July_August_2022.csv', index=False)